#### Team Member :
-  ⁠Cathy Cai    58990730⁠
-  Vineet Menon 90744863
-  ⁠Pranav Mehta 19184282
-  Chion Kim    87525523
-  ⁠Iqra Saeed   68400118

## a. Formulate (algebraically) an optimization problem to make a daily diet plan for each child, with the goal of minimizing the total cost of the food while satisfying the general nutritional requirements and additional considerations stated above.

#### Answer to (a)

### Definitions

- Let $x_i$ be the servings of each type of food in Table 4.
- Let $c_i$ be the cost of each type of food per serving.

### Table of Variables

| Variable Name        | Definition                                      |
|----------------------|-------------------------------------------------|
| $x_i$            | Number of servings for food $i$ in Table 4  |
| $c_i$            | Cost of each type of food $i$ per serving   |
| $\text{cal}_i$  | Calories per serving for food $i$           |
| $\text{sodi}_i$  | Sodium per serving for food $i$             |
| $\text{fi}_i$    | Fat per serving for food $i$                |
| $\text{carb}_i$ | Carbs per serving for food $i$              |
| $\text{fib}_i$  | Fiber per serving for food $i$              |
| $\text{p}_i$    | Protein per serving for food $i$            |
| $\text{va}_i$   | Vitamin A per serving for food $i$          |
| $\text{vc}_i$   | Vitamin C per serving for food $i$          |
| $\text{ca}_i$   | Calcium per serving for food $i$            |
| $\text{irn}_i$  | Iron per serving for food $i$               |

### Goal

Minimize the following formula:

$$
\text{Min} \left(\sum_{i=1}^{30} c_i \times x_i\right)
$$

### Constraints

#### Nutrient Intake Constraints

1. Caloric intake:
   $$
   1800 \leq \sum_{i=1}^{30} (\text{cal}_i \times x_i) \leq 2400
   $$

2. Fat intake:
   $$
   60 \leq \sum_{i=1}^{30} (\text{fi}_i \times x_i) \leq 95
   $$

3. Sodium intake:
   $$
   1200 \leq \sum_{i=1}^{30} (\text{sodi}_i \times x_i) \leq 2200
   $$

4. Carbohydrate intake:
   $$
   240 \leq \sum_{i=1}^{30} (\text{carb}_i \times x_i) \leq 400
   $$

5. Fiber intake:
   $$
   30 \leq \sum_{i=1}^{30} (\text{fib}_i \times x_i) \leq 35
   $$

6. Protein intake:
   $$
   40 \leq \sum_{i=1}^{30} (\text{p}_i \times x_i) \leq 55
   $$

7. Vitamin A intake:
   $$
   2000 \leq \sum_{i=1}^{30} (\text{va}_i \times x_i) \leq 6000
   $$

8. Vitamin C intake:
   $$
   45 \leq \sum_{i=1}^{30} (\text{vc}_i \times x_i) \leq 1200
   $$

9. Calcium intake:
   $$
   1300 \leq \sum_{i=1}^{30} (\text{ca}_i \times x_i) \leq 3000
   $$

10. Iron intake:
    $$
    8 \leq \sum_{i=1}^{30} (\text{irn}_i \times x_i) \leq 40
    $$

#### Balanced Diet Constraints

1. Protein balance:
   $$
   \text{p}_i \times x_i \leq 0.3 \times \sum_{i=1}^{30} (\text{pi}_i \times x_i) \quad \text{for every } i
   $$

2. Caloric balance:
   $$
   \text{cal}_i \times x_i \leq 0.3 \times \sum_{i=1}^{30} (\text{cali}_i \times x_i) \quad \text{for every } i
   $$


## b. Solve your formulation from part a) using Python/Gurobi to come up with a recommendation for the daily meal plan. Indicate the daily cost and the amount of each ingredient.

In [1]:
# Answer to (b)
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load the data from excel into python, then transposit the diet_goal matrix in order to have gurobi to successfully load the data.
diet_goal = pd.read_excel('food_data.xlsx', sheet_name='Sheet1', index_col=0)
food_data = pd.read_excel('food_data.xlsx', sheet_name='Sheet2', index_col=0)
diet_goal = diet_goal.T

# create the model
model = Model("DietPlanning")

# decision variables - quantity of each food item in servings
food_vars = {row.name: model.addVar(vtype=GRB.CONTINUOUS, name= row.name) for _, row in food_data.iterrows()}

# set objective, for (a) our goal is to minimize the cost of daily meal plan, while meeting all requirements for nutriention.

model.setObjective(quicksum(food_vars[i] * row['Cost ($/serving)'] for i, row in food_data.iterrows()), GRB.MINIMIZE)

# set constraints

for i, nutrient in enumerate(diet_goal.columns):
    model.addConstr(
        quicksum(food_vars[food] * row[nutrient] for food, row in food_data.iterrows()) >= diet_goal.iloc[0, i],
        name=f"Min_{nutrient}"
    )
    model.addConstr(
        quicksum(food_vars[food] * row[nutrient] for food, row in food_data.iterrows()) <= diet_goal.iloc[1, i],
        name=f"Max_{nutrient}"
    )

# add the constraint that no single food item should exceed 30% of the total calories or protein intake
total_calories = quicksum(food_vars[food] * row['Calories (kcal)'] for food, row in food_data.iterrows())
total_protein = quicksum(food_vars[food] * row['Protein (g)'] for food, row in food_data.iterrows())

for food, row in food_data.iterrows():
    model.addConstr(food_vars[food] * row['Calories (kcal)'] <= 0.3 * total_calories, name=f"CalorieLimit_{food}")
    model.addConstr(food_vars[food] * row['Protein (g)'] <= 0.3 * total_protein, name=f"ProteinLimit_{food}")

# optimize the model
model.optimize()

# display the results, displaying serving sizes for different food and rounded to 4 decimal places.
if model.status == GRB.OPTIMAL:
    print("\nOptimal daily meal plan:")
    for food, var in food_vars.items():
        if var.x > 0:
            print(f"{food}: {var.x.round(3)} Serving") #{food_data.loc[food, 'Serving']} for serving, you can consider split the serving column
    print(f"\nTotal Cost: ${model.objVal:.2f} per day")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-31
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 80 rows, 30 columns and 2326 nonzeros
Model fingerprint: 0x786ef467
Coefficient statistics:
  Matrix range     [6e-02, 2e+04]
  Objective range  [2e-02, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 6e+03]
Presolve removed 10 rows and 0 columns
Presolve time: 0.01s
Presolved: 70 rows, 40 columns, 2073 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.294375e+02   0.000000e+00      0s
      20    2.4310277e+00   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.431027707e+00

Optimal daily meal p

AttributeError: 'float' object has no attribute 'round'


### [ Answer to B ] Team's recommendation for the daily meal plan:
#### - Total Cost: $2.43 per day
#### - Food Items:
- White Rice: 0.157 Serving
- Spaghetti W/ Sauce: 0.611 Serving
- Potatoes, Baked: 3.149 Serving
- Oranges: 4.540 Serving
- Banana: 0.0941 Serving
- Pork: 0.567 Serving
- 2% Lowfat Milk: 2.037 Serving
- Skim Milk: 1.001 Serving



## C. Discuss whether you think the recommendation from part b) sounds reasonable for a single day of food intake for a child.  If not, suggest some changes you would make to the model to make the recommendation for the day more realistic.  Implement your change and discuss the new solution.


### [ Answer to C - Part 1/2 ]
#### While the recommended meal plan in part b) is mathematically optimal in terms of cost minimization and meeting nutritional requirements, it presents several practical challenges for a child’s daily intake. These challenges arise due to issues such as redundancy in food items, lack of variety, portion sizes,  and unrealistic consumption patterns. Specifically:


- Redundancy in Food Categories: The plan includes both '2% Lowfat Milk' and 'Skim Milk' in the same day, which is not typical for a child’s daily intake. Incorporating both 2% Lowfat Milk and Skim Milk in the same day complicates meal planning.


- Lack of Variety Across Meals: With a limited selection of food items, it would be challenging to create three distinct meals—breakfast, lunch, and dinner—without repeating the same items. For example, heavy reliance on potatoes and oranges would limit the variety needed to make meals more appealing and varied in taste and texture.


- Excessive Quantities of Certain Foods: The model suggests consuming over 3 servings of 'Baked Potatoes' and 4.5 servings of 'Oranges,' which is unrealistic for a child’s single-day intake. While the meal plan meets nutritional requirements, it may not be appetizing or practical to consume such large quantities of specific foods in one day.


- Impractical Portion Sizes: Some of the recommended servings, such as 0.157 servings of 'White Rice' or 0.0941 servings of 'Banana,' are not practical for preparation or consumption. These fractional servings would need to be rounded to feasible amounts, which could slightly alter the nutritional intake and cost.

#### The team introduced penalties for not incorporating a wider variety of foods into the objective function. We classified the food items into broader food groups to promote more variety in the daily meals, and we established constraints for the maximum and minimum number of food items in each food group.




| Meal Category | Foods                                                                                                                   |
|---------------|-------------------------------------------------------------------------------------------------------------------------|
| CARBS         | 'White Rice', 'Spaghetti W/ Sauce', 'Oatmeal', 'Potatoes, Baked', 'Corn', 'Wheat Bread', 'Couscous', 'White Bread', 'Macaroni, cooked', 'Bagels' |
| FRUIT         | 'Apple, Raw, w/Skin', 'Oranges', 'Banana', 'Kiwifruit, Raw, Fresh', 'Grapes'                                              |
| MILK          | '2% Lowfat Milk', 'Skim Milk'                                                                                            |
| PROTEIN       | 'Tofu', 'White Tuna in Water', 'Roasted Chicken', 'Pork', 'Turkey', 'Poached Eggs', 'Beef', 'Scrambled Eggs'             |
| VEGE          | 'Broccoli', 'Carrots, Raw', 'Tomato,Red,Ripe,Raw', 'Peppers, Sweet, Raw', 'Lettuce, Iceberg,Raw'                         |


####  To address the issue of impractical or excessive servings of a few food items, we added a constraint that specifies a range of servings for food items from 0.5 to 2.
#### This adjustment encourages a more diverse, practical and balanced meal plan.

#### A minimum of 3 items from the carbohydrate food group and 2 items from the protein food group must be included in the meal plan.
#### To limit the meal plan to only one type of milk, we added a constraint that the amount of milk should be equal to or less than one. Similarly, for the fruit food group, we added a constraint to limit the variety of fruits to fewer than 2. However, these milk and fruit constraints resulted in an infeasible solution, and we had to relax both constraints in order to achieve an optimal solution for the daily meal plan.


#### Below is the summary of objective function, new constraints and revised code implementing this approach:


| Category        | Content                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|-----------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Objective**   | Minimize the total cost while maximizing the diversity of food intake. This dual objective aims to achieve cost efficiency alongside nutritional variety by ensuring a balanced inclusion of different types of food items each day.                                                                                                                                                                                                                                                                           |
| **Constraints** | **Serving size:** Each type of food should have a serving size that is no less than 0.4 and no more than 2 servings.<br>**Food Variety:** Ensure a minimum inclusion of carbohydrate and protein food types to promote dietary diversity.<br>**Storage Concerns:** Implement a constraint on the maximum number of fruit types due to their perishability.<br> Additionally, limit the total number of different food types in any given meal plan to 12, to manage complexity and ensure practicality in meal preparation. |

This table uses Markdown to format the explanation of the model’s objective and constraints as per your structure, suitable for inclusion in a Jupyter Notebook.


In [ ]:
# part c

# The same as model displayed in part b, we load the data into the model, perform data cleansing to suit our request.
diet_goal = pd.read_excel('food_data.xlsx', sheet_name='Sheet1', index_col=0)
food_data = pd.read_excel('food_data.xlsx', sheet_name='Sheet2', index_col=0)
diet_goal = diet_goal.T
# Create the model
model = Model("DietPlanning")


# Creating Decision variables: quantity of each food item in servings
food_vars = {row.name: model.addVar(lb = 0, vtype=GRB.CONTINUOUS, name=row.name) for _, row in food_data.iterrows()}

# Classification sets for food types (assuming these classifications are in your dataset)
carb_foods = food_data[food_data['Meal Category'] == 'CARBS'].index.tolist()
protein_foods = food_data[food_data['Meal Category'] == 'PROTEIN'].index.tolist()
fruit_foods = food_data[food_data['Meal Category'] == 'FRUIT'].index.tolist()


# Adding Binary variables to indicate if a food item is included in the diet.
is_included = {food: model.addVar(vtype=GRB.BINARY, name=f"Included_{food}") for food in food_data.index}



# Set objective. For goal programming we take 2 goal into consideration: minimizing total cost and maximize variability of food intake everyday.
# After discussion, our team decided to assign 0.4 weight to cost minimizing and 0.6 weight to food-variability maximization.
# For this model, we consider food variability to be a more important factor.
model.setObjective(0.7 * quicksum(food_vars[i] * row['Cost ($/serving)'] for i, row in food_data.iterrows())
                   - 0.3 * quicksum(is_included[food] for food in food_vars), GRB.MINIMIZE)

# Set nutritional constraints
for i, nutrient in enumerate(diet_goal.columns):
    model.addConstr(
        quicksum(food_vars[food] * row[nutrient] for food, row in food_data.iterrows()) >= diet_goal.iloc[0, i],
        name=f"Min_{nutrient}"
    )
    model.addConstr(
        quicksum(food_vars[food] * row[nutrient] for food, row in food_data.iterrows()) <= diet_goal.iloc[1, i],
        name=f"Max_{nutrient}"
    )
# Link binary variables with food serving variables.
# Meanwhile adding constraints on serving size for each food to be no less than 0.4 servings per day and no more than 2 servings per day.
for food, var in food_vars.items():
    model.addConstr(var >= 0.4 * is_included[food], name=f"MinServing_{food}")
    model.addConstr(var <= 2 * is_included[food], name=f"MaxServing_{food}")




# Add the constraint that no single food item should exceed 30% of the total calories or protein intake
total_calories = quicksum(food_vars[food] * row['Calories (kcal)'] for food, row in food_data.iterrows())
total_protein = quicksum(food_vars[food] * row['Protein (g)'] for food, row in food_data.iterrows())

for food, row in food_data.iterrows():
    model.addConstr(food_vars[food] * row['Calories (kcal)'] <= 0.3 * total_calories, name=f"CalorieLimit_{food}")
    model.addConstr(food_vars[food] * row['Protein (g)'] <= 0.3 * total_protein, name=f"ProteinLimit_{food}")


# Add constraints for variety in food types. Setting contraints on minimum types of carb food and protein food for variability
# Setting a constraint on maximum types of fruit food as durability of fruit is shorter than other types of food
# making fruit more difficult to store.
# Out of concerns to storage, we limited the total types of food in one meal to be no more than 12
model.addConstr(quicksum(is_included[food] for food in carb_foods) >= 3, name="MinCarbs")
model.addConstr(quicksum(is_included[food] for food in protein_foods) >=2, name="MinProteins")
model.addConstr(quicksum(is_included[food] for food in fruit_foods) <= 3, name="MinFruits")
model.addConstr(quicksum(is_included[food] for food in food_vars) <= 12, name="MaxFoods")


# Optimize the model
model.optimize()

# If the model is infeasible, compute the IIS to diagnose the issue
if model.status == GRB.INFEASIBLE:
    print("\nModel is infeasible. Identifying conflicting constraints...")
    model.computeIIS()
    model.write("dietplan_ilp.ilp")

    # Print the constraints that are causing the issue
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Infeasible constraint: {c.constrName}")

# Display the results
if model.status == GRB.OPTIMAL:
    total_cost_plan = sum(food_vars[food].x * row['Cost ($/serving)'] for food, row in food_data.iterrows())
    print("\nOptimal daily meal plan:")
    for food, var in food_vars.items():
        if var.x > 0:
            print(f"{food}: {var.x:.2f} Servings")
    print(f"\nTotal Cost: ${total_cost_plan:.2f} per day")

ModuleNotFoundError: No module named 'gurobipy'


### [ Answer to C - Part 2/2 ]
#### To make the meal recommendation for the day more realistic, we made the following changes:


#### - Total Cost: $2.80 per day
#### - Food Items:
- Spaghetti W/ Sauce: 0.40 Servings
- Apple, Raw, w/Skin: 2.00 Servings
- Potatoes, Baked: 1.44 Servings
- Oranges: 2.00 Servings
- Banana: 2.00 Servings
- Wheat Bread: 0.40 Servings
- White Bread: 0.40 Servings
- Lettuce, Iceberg, Raw: 0.40 Servings
- Pork: 0.64 Servings
- Turkey: 0.40 Servings
- Beef: 0.40 Servings
- 2% Lowfat Milk: 2.00 Servings
- Skim Milk: 1.44 Servings

#### The team concluded that the current constraints for minimum nutrients are too restrictive to further improve the daily meal plan. The 2% low-fat milk and skim milk contribute 51%, 43%, and 79% to the total protein, vitamin A, and calcium, respectively. Additionally, all three fruits contribute 48%, 63%, and 80% to carbohydrates, fiber, and vitamin C.

#### Given that these food items are major contributors to 6 out of 10 nutrients in the model simultaneously, their inclusion in the meal plan is crucial for meeting dietary requirements. Further restricting these items results in an infeasible solution for the model.


## D. Whereas part c) asked you to consider whether the meal plan for the one day is sensible or not and how you might get to a reasonable plan for even one day.  However, Principal Harper also knows that kids don’t like to eat the same exact thing every day, so for part d) she asks you to come up with a reasonable dietary plan for each day of an entire week.  Present a 7-day dietary plan, and indicate any model changes that went into constructing each day’s plan.

### [ Answer to D - Part 1/2 ]
#### 7-Day Dietary Plan Based on the Given 'Daily Nutritional Requirements'

#### Day 1
#### - Total Cost: $2.80 per day
#### - Food Items:
- White Rice: 0.30 Servings
- Spaghetti W/ Sauce: 0.37 Servings
- Apple, Raw, w/Skin: 1.70 Servings
- Potatoes, Baked: 1.60 Servings
- Oranges: 2.00 Servings
- Banana: 2.00 Servings
- Wheat Bread: 0.30 Servings
- White Bread: 0.30 Servings
- Bagels: 0.30 Servings
- Lettuce, Iceberg,Raw: 0.30 Servings
- Grapes: 0.30 Servings
- Pork: 0.58 Servings
- Beef: 0.30 Servings
- 2% Lowfat Milk: 2.00 Servings
- Skim Milk: 1.49 Servings

#### Day 2
#### - Total Cost: $3.78 per day
#### - Food Items:
- Spaghetti W/ Sauce: 0.75 Servings
- Apple, Raw, w/Skin: 2.00 Servings
- Potatoes, Baked: 1.51 Servings
- Oranges: 2.00 Servings
- Kiwifruit, Raw, Fresh: 2.00 Servings
- Tomato,Red,Ripe,Raw: 0.30 Servings
- Pork: 0.73 Servings
- Turkey: 0.30 Servings
- 2% Lowfat Milk: 2.00 Servings
- Skim Milk: 1.31 Servings

#### Day 3
#### - Total Cost: $2.88 per day
#### - Food Items:
- Spaghetti W/ Sauce: 0.55 Servings
- Apple, Raw, w/Skin: 1.95 Servings
- Potatoes, Baked: 1.94 Servings
- Oranges: 2.00 Servings
- Banana: 2.00 Servings
- Peppers, Sweet, Raw: 0.30 Servings
- Pork: 0.54 Servings
- Poached Eggs: 0.30 Servings
- 2% Lowfat Milk: 2.00 Servings
- Skim Milk: 1.45 Servings

#### Day 4
#### - Total Cost: $3.77 per day
#### - Food Items:
- Spaghetti W/ Sauce: 0.55 Servings
- Apple, Raw, w/Skin: 2.00 Servings
- Potatoes, Baked: 0.83 Servings
- Oranges: 2.00 Servings
- Banana: 2.00 Servings
- Kiwifruit, Raw, Fresh: 1.71 Servings
- Corn: 0.30 Servings
- Couscous: 0.30 Servings
- Macaroni, cooked: 0.30 Servings
- Pork: 0.58 Servings
- Scrambled Eggs: 0.30 Servings
- 2% Lowfat Milk: 2.00 Servings
- Skim Milk: 1.39 Servings

#### Day 5
#### - Total Cost: $2.95 per day
#### - Food Items:
- Spaghetti W/ Sauce: 0.59 Servings
- Apple, Raw, w/Skin: 1.96 Servings
- Potatoes, Baked: 1.87 Servings
- Oranges: 2.00 Servings
- Banana: 2.00 Servings
- Kiwifruit, Raw, Fresh: 0.30 Servings
- Tofu: 0.30 Servings
- Pork: 0.53 Servings
- 2% Lowfat Milk: 2.00 Servings
- Skim Milk: 1.34 Servings

#### Day 6
#### - Total Cost: $3.50 per day
#### - Food Items:
- Spaghetti W/ Sauce: 0.52 Servings
- Apple, Raw, w/Skin: 2.00 Servings
- Potatoes, Baked: 1.91 Servings
- Oranges: 2.00 Servings
- Kiwifruit, Raw, Fresh: 1.84 Servings
- Pork: 0.76 Servings
- Turkey: 0.30 Servings
- 2% Lowfat Milk: 2.00 Servings
- Skim Milk: 1.35 Servings

#### Day 7
#### - Total Cost: $3.66 per day
#### - Food Items:
- Spaghetti W/ Sauce: 0.66 Servings
- Apple, Raw, w/Skin: 2.00 Servings
- Potatoes, Baked: 1.66 Servings
- Oranges: 2.00 Servings
- Kiwifruit, Raw, Fresh: 2.00 Servings
- Pork: 0.70 Servings
- Beef: 0.30 Servings
- 2% Lowfat Milk: 2.00 Servings
- Skim Milk: 1.35 Servings

<br><br>

#### As shown in the results, despite our efforts to meet the 'Daily Nutritional Requirements', we were unable to eliminate 'Spaghetti' from the daily meal plan. This is due to the minimum sodium constraint, as there were no other foods available that could meet the sodium requirement without exceeding the limits for other nutrients like protein. While we managed to introduce some variety into the 7-day plan, children will still have to eat 'Spaghetti' every single day.

In [ ]:
# part d - part(1/2)
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load the data
diet_goal = pd.read_excel('food_data.xlsx', sheet_name='Sheet1', index_col=0)
food_data = pd.read_excel('food_data.xlsx', sheet_name='Sheet2', index_col=0)
diet_goal = diet_goal.T

# Create the model
model = Model("DietPlanning")

# Decision variables - quantity of each food item in servings
food_vars = {row.name: model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=row.name) for _, row in food_data.iterrows()}

# Classification sets for food types (assuming these classifications are in your dataset)
carb_foods = food_data[food_data['Meal Category'] == 'CARBS'].index.tolist()
protein_foods = food_data[food_data['Meal Category'] == 'PROTEIN'].index.tolist()
milk_foods = food_data[food_data['Meal Category'] == 'MILK'].index.tolist()
fruit_foods = food_data[food_data['Meal Category'] == 'FRUIT'].index.tolist()

# Binary variables to indicate if a food item is included in the diet
is_included = {food: model.addVar(vtype=GRB.BINARY, name=f"Included_{food}") for food in food_data.index}

# Link binary variables with food serving variables
for food, var in food_vars.items():
    model.addConstr(var >= 0.3 * is_included[food], name=f"MinServing_{food}")
    model.addConstr(var <= 2 * is_included[food], name=f"MaxServing_{food}")

# Add constraints for variety in food types
model.addConstr(quicksum(is_included[food] for food in carb_foods) >= 2, name="MinCarbs")
model.addConstr(quicksum(is_included[food] for food in protein_foods) >= 2, name="MinProteins")
model.addConstr(quicksum(is_included[food] for food in fruit_foods) <= 4, name="MinFruits")
model.addConstr(quicksum(is_included[food] for food in food_vars) <= 15, name="MaxFoods")
# Set objective: minimize the total cost of the diet
model.setObjective(0.4 * quicksum(food_vars[i] * row['Cost ($/serving)'] for i, row in food_data.iterrows())
                   - 0.6 * quicksum(is_included[food] for food in food_vars), GRB.MINIMIZE)

# Set nutritional constraints
for i, nutrient in enumerate(diet_goal.columns):
    model.addConstr(
        quicksum(food_vars[food] * row[nutrient] for food, row in food_data.iterrows()) >= diet_goal.iloc[0, i],
        name=f"Min_{nutrient}"
    )
    model.addConstr(
        quicksum(food_vars[food] * row[nutrient] for food, row in food_data.iterrows()) <= diet_goal.iloc[1, i],
        name=f"Max_{nutrient}"
    )

# Add the constraint that no single food item should exceed 30% of the total calories or protein intake
total_calories = quicksum(food_vars[food] * row['Calories (kcal)'] for food, row in food_data.iterrows())
total_protein = quicksum(food_vars[food] * row['Protein (g)'] for food, row in food_data.iterrows())

for food, row in food_data.iterrows():
    model.addConstr(food_vars[food] * row['Calories (kcal)'] <= 0.3 * total_calories, name=f"CalorieLimit_{food}")
    model.addConstr(food_vars[food] * row['Protein (g)'] <= 0.3 * total_protein, name=f"ProteinLimit_{food}")

# Set Gurobi parameters to search for multiple solutions
model.setParam(GRB.Param.PoolSearchMode, 2)  # Allow Gurobi to find multiple solutions
model.setParam(GRB.Param.PoolSolutions, 10)  # Store up to 10 solutions for flexibility

# Optimize the model
model.optimize()

# If the model is infeasible, compute the IIS to diagnose the issue
if model.status == GRB.INFEASIBLE:
    print("\nModel is infeasible. Identifying conflicting constraints...")
    model.computeIIS()
    model.write("dietplan_ilp.ilp")

    # Print the constraints that are causing the issue
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Infeasible constraint: {c.constrName}")

# Display the top 7 best solutions if the model is optimal or suboptimal
if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
    solution_count = min(model.SolCount, 7)  # We want up to 7 solutions
    print(f"\nNumber of solutions found: {solution_count}")

    for sol_num in range(solution_count):
        # Set the solution number to retrieve
        model.setParam(GRB.Param.SolutionNumber, sol_num)
used_foods = set()
# Display the top 7 best solutions if the model is optimal or suboptimal
# Display the top 7 best solutions if the model is optimal or suboptimal
if model.status == GRB.OPTIMAL:
    solution_count = min(model.SolCount, 7)  # We want up to 7 solutions
    print(f"\nNumber of solutions found: {solution_count}")

    for sol_num in range(solution_count):
        # Set the solution number to retrieve
        model.setParam(GRB.Param.SolutionNumber, sol_num)

        # Retrieve the total cost for the current solution
        total_cost = quicksum(food_vars[i].Xn * row['Cost ($/serving)'] for i, row in food_data.iterrows()).getValue()
        print(f"\nSolution {sol_num + 1}:")
        print("Optimal daily meal plan:")
        current_solution_foods = set()
        for food, var in food_vars.items():
            if var.Xn > 0:
                print(f"{food}: {var.Xn:.2f} Servings")
                current_solution_foods.add(food)
        print(f"Total Cost: ${total_cost:.2f} per day")

        # Add constraint to exclude current solution foods from future solutions
        used_foods = used_foods.union(current_solution_foods)
        print(used_foods)
        model.setObjective(0.01 * quicksum(food_vars[i] * row['Cost ($/serving)'] for i, row in food_data.iterrows())
                   - 0.9 * quicksum(is_included[food] for food in food_vars) + 10000*quicksum(is_included[food] for food in used_foods), GRB.MINIMIZE)

        # Re-optimize the model for the next solution
        model.optimize()

### [ Answer to D - Part 2/2 ]
#### Principal Harper expressed concerns that children don’t enjoy eating the same meal every day. To address this, we developed a 7-day dietary plan with relaxed constraints, as shown below:

#### Day 1
#### - Total Cost: $1.98 per day
#### - Food Items:
- White Rice: 0.70 Servings
- Potatoes, Baked: 2.00 Servings
- Oranges: 0.30 Servings
- Banana: 2.00 Servings
- Carrots, Raw: 0.30 Servings
- Wheat Bread: 2.00 Servings
- White Bread: 2.00 Servings
- Macaroni, cooked: 0.76 Servings
- Bagels: 1.03 Servings
- Lettuce, Iceberg,Raw: 0.30 Servings
- Pork: 0.70 Servings
- Poached Eggs: 0.30 Servings
- Beef: 0.30 Servings
- Scrambled Eggs: 0.30 Servings
- Skim Milk: 1.69 Servings

<br><br>

### Conclusion
#### Even with relaxed constraints, the limitations remain significant. In this 7-day plan, variety was introduced at the expense of not fulfilling the minimum requirement for calcium. Despite this compromise, we were only able to create one day without spaghetti.
#### The team recommends two possible actions for Principal Harper:
1. Hire a skilled cook who can creatively prepare more varied meals using the existing food items provided by the model.
2. Expand the list of available food items to give the model more flexibility in creating varied daily combinations.

<br>

#### Without increasing the diversity of food options, it will be challenging to introduce meaningful variety in meals, even if some nutritional requirements are sacrificed.

In [ ]:
# part d - part(2/2)
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load the data
diet_goal = pd.read_excel('food_data.xlsx', sheet_name='Sheet1', index_col=0)
food_data = pd.read_excel('food_data.xlsx', sheet_name='Sheet2', index_col=0)
diet_goal = diet_goal.T

# Create the model
model = Model("DietPlanning")

# Decision variables - quantity of each food item in servings
food_vars = {row.name: model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=row.name) for _, row in food_data.iterrows()}
Calu = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name='calcium_under')
Calo = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name='calcium_over')


# Classification sets for food types (assuming these classifications are in your dataset)
carb_foods = food_data[food_data['Meal Category'] == 'CARBS'].index.tolist()
protein_foods = food_data[food_data['Meal Category'] == 'PROTEIN'].index.tolist()
milk_foods = food_data[food_data['Meal Category'] == 'MILK'].index.tolist()
fruit_foods = food_data[food_data['Meal Category'] == 'FRUIT'].index.tolist()

# Binary variables to indicate if a food item is included in the diet
is_included = {food: model.addVar(vtype=GRB.BINARY, name=f"Included_{food}") for food in food_data.index}

# Link binary variables with food serving variables
for food, var in food_vars.items():
    model.addConstr(var >= 0.3 * is_included[food], name=f"MinServing_{food}")
    model.addConstr(var <= 2 * is_included[food], name=f"MaxServing_{food}")

# Add constraints for variety in food types
model.addConstr(quicksum(is_included[food] for food in carb_foods) >= 2, name="MinCarbs")
model.addConstr(quicksum(is_included[food] for food in protein_foods) >= 2, name="MinProteins")
model.addConstr(quicksum(is_included[food] for food in fruit_foods) <= 4, name="MinFruits")
model.addConstr(quicksum(is_included[food] for food in milk_foods) <= 1, name="MinFruits")
model.addConstr(quicksum(is_included[food] for food in food_vars) <= 15, name="MaxFoods")
# Set objective: minimize the total cost of the diet
model.setObjective(0.4 * quicksum(food_vars[i] * row['Cost ($/serving)'] for i, row in food_data.iterrows())
                   - 0.6 * quicksum(is_included[food] for food in food_vars) + 0.0001 * Calu + 0.0001*Calo , GRB.MINIMIZE)

# Set nutritional constraints
for i, nutrient in enumerate(diet_goal.columns):
    nutrient_sum = quicksum(food_vars[food] * row[nutrient] for food, row in food_data.iterrows())

    if nutrient == 'Calcium (mg)':
        model.addConstr(
            nutrient_sum + Calu >= diet_goal.iloc[0, i] ,  # 例如，蛋白质的最小值增加10%
            name=f"Min_{nutrient}"
        )
        model.addConstr(
            nutrient_sum - Calo <= diet_goal.iloc[1, i],  # 例如，蛋白质的最大值减少10%
            name=f"Max_{nutrient}"
        )
    else:
        model.addConstr(
            nutrient_sum >= diet_goal.iloc[0, i],
            name=f"Min_{nutrient}"
        )
        model.addConstr(
            nutrient_sum <= diet_goal.iloc[1, i],
            name=f"Max_{nutrient}"
        )

# Add the constraint that no single food item should exceed 30% of the total calories or protein intake
total_calories = quicksum(food_vars[food] * row['Calories (kcal)'] for food, row in food_data.iterrows())
total_protein = quicksum(food_vars[food] * row['Protein (g)'] for food, row in food_data.iterrows())

for food, row in food_data.iterrows():
    model.addConstr(food_vars[food] * row['Calories (kcal)'] <= 0.3 * total_calories, name=f"CalorieLimit_{food}")
    model.addConstr(food_vars[food] * row['Protein (g)'] <= 0.3 * total_protein, name=f"ProteinLimit_{food}")

# Set Gurobi parameters to search for multiple solutions
model.setParam(GRB.Param.PoolSearchMode, 2)  # Allow Gurobi to find multiple solutions
model.setParam(GRB.Param.PoolSolutions, 10)  # Store up to 10 solutions for flexibility

# Optimize the model
model.optimize()

# If the model is infeasible, compute the IIS to diagnose the issue
if model.status == GRB.INFEASIBLE:
    print("\nModel is infeasible. Identifying conflicting constraints...")
    model.computeIIS()
    model.write("dietplan_ilp.ilp")

    # Print the constraints that are causing the issue
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Infeasible constraint: {c.constrName}")

# Display the top 7 best solutions if the model is optimal or suboptimal
if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
    solution_count = min(model.SolCount, 7)  # We want up to 7 solutions
    print(f"\nNumber of solutions found: {solution_count}")

    for sol_num in range(solution_count):
        # Set the solution number to retrieve
        model.setParam(GRB.Param.SolutionNumber, sol_num)
used_foods = set()
# Display the top 7 best solutions if the model is optimal or suboptimal
# Display the top 7 best solutions if the model is optimal or suboptimal
if model.status == GRB.OPTIMAL:
    solution_count = min(model.SolCount, 7)  # We want up to 7 solutions
    print(f"\nNumber of solutions found: {solution_count}")

    for sol_num in range(solution_count):
        # Set the solution number to retrieve
        model.setParam(GRB.Param.SolutionNumber, sol_num)

        # Retrieve the total cost for the current solution
        total_cost = quicksum(food_vars[i].Xn * row['Cost ($/serving)'] for i, row in food_data.iterrows()).getValue()
        print(f"\nSolution {sol_num + 1}:")
        print("Optimal daily meal plan:")
        current_solution_foods = set()
        for food, var in food_vars.items():
            if var.Xn > 0:
                print(f"{food}: {var.Xn:.2f} Servings")
                current_solution_foods.add(food)
        print(f"Total Cost: ${total_cost:.2f} per day")

        # Add constraint to exclude current solution foods from future solutions
        used_foods = used_foods.union(current_solution_foods)
        print(used_foods)
        print(Calu.Xn)
        print(Calo.Xn)
        model.setObjective(0.01 * quicksum(food_vars[i] * row['Cost ($/serving)'] for i, row in food_data.iterrows())
                   - 0.9 * quicksum(is_included[food] for food in food_vars) +  0.0001 * Calu + 0.0001*Calo + 10000*quicksum(is_included[food] for food in used_foods), GRB.MINIMIZE)

        # Re-optimize the model for the next solution
        model.optimize()